In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# NLP Imports 
# set up spacy
import spacy

# get stopwords from NLTK
from nltk.corpus import stopwords
stopwords = stopwords.words("english")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

In [2]:
raw_sake = pd.read_json('prepared_data.json')
raw_sake.head()

,url,name,type,price,prefecture,smv,acidity,kw_word,kw_wines,kw_beer,kw_foods,description
0,https://www.truesake.com/collections/all/produ...,"Dassai 45 Junmai Daiginjo ""Otter Festival""",junmai daiginjo,30,yamaguchi,3.0,1.4,chunky,pinot noir chewy whites,ales,mushroom risotto tempura fried chicken caviar ...,this sake has an older brother daiginjo that h...
1,https://www.truesake.com/collections/all/produ...,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",junmai ginjo,33,niigata,1.0,1.6,citrus,sharp whites dry reds,ales ambers,suitable for light and stronger flavored dishe...,true sake owner beau timken recorded kikusui a...
2,https://www.truesake.com/collections/all/produ...,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",junmai daiginjo,49,shizuoka,0.0,1.4,popular,elegant reds complex whites,crisp ales,cuisines that like big white wines,talk about a nice aroma package on this ultrap...
3,https://www.truesake.com/collections/all/produ...,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",tokubetsu junmai,9,yamanashi,3.0,1.6,smooth,pinot noir slippery whites,gentle ales,none,the nose on this awesome looking catch cup is ...
4,https://www.truesake.com/collections/all/produ...,"Kenbishi Kuromatsu Honjozo ""Black Pine""",honjozo,40,hyogo,0.5,1.5,rich,deep reds fat whites,stouts,perfect for pub food salty and savory and grea...,the nose on this extremely famous honjozo from...


In [ ]:
class SakeRecommender:
    
    def __init__(self, preprocessed_sake_data):
        
        self.data = preprocessed_sake_data

        
        
    def recommend_sake(self, liked_sakes: list, num_recs=5, metric='cosine') -> list:
        
        self.sake = self.data[[
                               'name', 
                               'url', 
                               'price', 
                               'smv', 
                               'acidity', 
                               'type', 
                               'prefecture', 
                            ]]

        self.sake = pd.get_dummies(data=self.sake, 
                                   columns=['type', 'prefecture'])
         
        self.pca = PCA(n_components=1)

        self.df_pca = self.pca.fit_transform(self.sake.iloc[:, 5:])
        
        self.df = self.sake[['name', 'url', 'price', 'smv', 'acidity']]
        
        self.df['pca_feats'] = self.df_pca
        
        
        
        self.liked_sakes = liked_sakes
        self.num_recs = num_recs
        self.metric = metric
    
        self.dists = pairwise_distances(self.df.iloc[:, 2:], metric=self.metric)
        self.dists = pd.DataFrame(data=self.dists, index=self.df['name'], columns=self.df['name'])

        self.sake_summed = self.dists[self.liked_sakes].sum(axis=1)
        self.sake_summed = self.sake_summed.sort_values(ascending=True)


        self.ranked_sakes = self.sake_summed.index[~self.sake_summed.index.isin(self.liked_sakes)]
        self.ranked_sakes = self.ranked_sakes.tolist()

        self.recommendations = self.ranked_sakes[:self.num_recs]

        self.df_recs = self.data.copy()
        self.df_recs['rec_label'] = np.where(self.df_recs.name.isin(self.liked_sakes), 'Liked',
                                       np.where(self.df_recs.name.isin(self.recommendations), 'Recommended',
                                               'Other'))

        self.df_recs = self.df_recs[self.df_recs['rec_label'].isin(['Liked', 'Recommended'])]
        
        self.df_recs = self.df_recs[['rec_label', 
                                    'name',
                                    'type',
                                    'price',
                                    'prefecture',
                                    'smv',
                                    'acidity',
                                    'kw_word',
                                    'kw_wines',
                                    'kw_beer',
                                    'description',
                                    'url']]
        
        

        return self.df_recs

In [ ]:
s = SakeRecommender(raw_sake)

In [ ]:
sake_i_like = [
    'DEN Junmai Nama',
    'Tengumai Yamahai Junmai "Dance of the Demon"'
              ]

s.recommend_sake(sake_i_like, num_recs=3)

In [ ]:
s.df.head()

In [3]:
sake = raw_sake[[
           'name', 
           'url', 
           'price', 
           'smv', 
           'acidity', 
           'type', 
           'prefecture', 
           'kw_word', 
           'kw_wines', 
           'kw_beer', 
           'kw_foods', 
           'description'
          ]]

sake.head()

,name,url,price,smv,acidity,type,prefecture,kw_word,kw_wines,kw_beer,kw_foods,description
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,junmai daiginjo,yamaguchi,chunky,pinot noir chewy whites,ales,mushroom risotto tempura fried chicken caviar ...,this sake has an older brother daiginjo that h...
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,junmai ginjo,niigata,citrus,sharp whites dry reds,ales ambers,suitable for light and stronger flavored dishe...,true sake owner beau timken recorded kikusui a...
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",https://www.truesake.com/collections/all/produ...,49,0.0,1.4,junmai daiginjo,shizuoka,popular,elegant reds complex whites,crisp ales,cuisines that like big white wines,talk about a nice aroma package on this ultrap...
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,tokubetsu junmai,yamanashi,smooth,pinot noir slippery whites,gentle ales,none,the nose on this awesome looking catch cup is ...
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",https://www.truesake.com/collections/all/produ...,40,0.5,1.5,honjozo,hyogo,rich,deep reds fat whites,stouts,perfect for pub food salty and savory and grea...,the nose on this extremely famous honjozo from...


In [4]:
disabled_components = ['parser', 'tagger', 'ner']
nlp = spacy.load('en_core_web_sm', disable=disabled_components)

In [5]:
## Lemmatize!

def lemmatize(doc: str) -> str:
    
    # Set up spacy 
    disabled_components = ['parser', 'tagger', 'ner']
    nlp = spacy.load('en_core_web_sm', disable=disabled_components)
    
    # Lemmatize doc
    doc = nlp(doc)
    lemma_list = [str(tok.lemma_) for tok in doc if tok.text not in stopwords]
    lem_string = " ".join(lemma_list)
    return lem_string

In [6]:
df = sake.copy()

# df['kw_foods'] = df['kw_foods'].map(lemmatize)
# df['description'] = df['description'].map(lemmatize)

In [7]:
s = 'this is a sample'

lemmatize(s)

'sample'

In [8]:
df['text'] = df['type'] + ' ' + \
            df['prefecture'] + ' ' + \
            df['kw_word'] + ' ' + \
            df['kw_wines'] + ' ' + \
            df['kw_beer'] + ' ' + \
            df['kw_foods'] + ' ' + \
            df['description']

df.drop(columns=['type', 
                 'prefecture', 
                 'kw_word', 
                 'kw_wines', 
                 'kw_beer', 
                 'kw_foods', 
                 'description'], inplace=True)

# df['text'] = df['text'].map(lemmatize)

In [9]:
df.head()

,name,url,price,smv,acidity,text
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,junmai daiginjo yamaguchi chunky pinot noir c...
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,junmai ginjo niigata citrus sharp whites dry r...
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",https://www.truesake.com/collections/all/produ...,49,0.0,1.4,junmai daiginjo shizuoka popular elegant reds ...
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,tokubetsu junmai yamanashi smooth pinot noir s...
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",https://www.truesake.com/collections/all/produ...,40,0.5,1.5,honjozo hyogo rich deep reds fat whites stouts...


In [11]:
# Takes about 3 mins to run
df['text'] = df['text'].map(lemmatize)

In [12]:
df.head()

,name,url,price,smv,acidity,text
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,junmai daiginjo yamaguchi chunky pinot noir ...
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,junmai ginjo niigata citrus sharp white dry re...
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",https://www.truesake.com/collections/all/produ...,49,0.0,1.4,junmai daiginjo shizuoka popular elegant red c...
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,tokubetsu junmai yamanashi smooth pinot noir s...
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",https://www.truesake.com/collections/all/produ...,40,0.5,1.5,honjozo hyogo rich deep red fat white stout pe...


In [13]:
df.text[1]

'junmai ginjo niigata citrus sharp white dry red ale ambers suitable light strong flavor dish dim sum chicken pork mild spice creamy cheese fresh fruit true sake owner beau timken record kikusui \ufeff1 entry personal journal 200 sake would late purchase kikusui \ufeff1 case sake thus kikusui hold special place shelve ginjo slightly fruity nose hint citrus persimmon kikusui confidently smooth begin gradual thicken viscosity end slightly salty aftertaste dry sharp smooth accord ginjo sit good mouth tease sweet fruit tone dry finish kikusui ferment low temperature long ginjos rid sake sometimes heavy characteristic junmai thus give overall light dry complexion gin vodka drinker enjoy'

In [14]:
# Vectorize
v = TfidfVectorizer(max_features=1000)
text_tfidf = v.fit_transform(df['text'])
txt = text_tfidf.toarray()

# Create tfidf_df
tfidf_df = pd.DataFrame(txt, columns=v.get_feature_names())

In [15]:
tfidf_df.head()

,10,12,17,18,20,23,40,50,75,80,...,yellow,yes,yet,yogurt,young,yup,yuzu,zesty,zins,zone
0,0.0,0.0,0.000000,0.0,0.0,0.157475,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.164374,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df = pd.concat([df, tfidf_df], axis=1)

df.drop(columns=['text'], inplace=True)

In [17]:
df.head()

,name,url,price,smv,acidity,10,12,17,18,20,...,yellow,yes,yet,yogurt,young,yup,yuzu,zesty,zins,zone
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",https://www.truesake.com/collections/all/produ...,49,0.0,1.4,0.0,0.0,0.164374,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",https://www.truesake.com/collections/all/produ...,40,0.5,1.5,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
pca = PCA(n_components=1)

df_pca = pca.fit_transform(df.iloc[:, 5:])

In [19]:
df = df[[
        'name', 
        'url', 
        'price', 
        'smv', 
        'acidity'
        ]]

In [20]:
# df['pca_1'] = df_pca[:, 0]
# df['pca_2'] = df_pca[:, 1]

df['pca_feats'] = df_pca

In [21]:
df.head()

,name,name,url,price,smv,smv,acidity,acidity,pca_feats
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",0.0,https://www.truesake.com/collections/all/produ...,30,3.0,0.0,1.4,0.000000,-0.060901
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",0.0,https://www.truesake.com/collections/all/produ...,33,1.0,0.0,1.6,0.000000,-0.029611
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",0.0,https://www.truesake.com/collections/all/produ...,49,0.0,0.0,1.4,0.087531,-0.105122
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",0.0,https://www.truesake.com/collections/all/produ...,9,3.0,0.0,1.6,0.000000,0.037787
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",0.0,https://www.truesake.com/collections/all/produ...,40,0.5,0.0,1.5,0.000000,-0.061783


In [22]:
# Getting this weird magic duplicate 'name' column?

df.columns = ['name', 
              'n2', 
              'url', 
              'price', 
              'smv', 
              'smv', 
              'acidity', 
              'n2',
              'pca_feats'
             ]

df.drop(columns=['n2'], inplace=True)

In [23]:
df.head()

,name,url,price,smv,smv,acidity,pca_feats
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,0.0,1.4,-0.060901
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,0.0,1.6,-0.029611
2,"Wakatake Onikoroshi Junmai Daiginjo ""Demon Sla...",https://www.truesake.com/collections/all/produ...,49,0.0,0.0,1.4,-0.105122
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,0.0,1.6,0.037787
4,"Kenbishi Kuromatsu Honjozo ""Black Pine""",https://www.truesake.com/collections/all/produ...,40,0.5,0.0,1.5,-0.061783


In [24]:
# import distance methods

dist = pairwise_distances(df.iloc[:, 2:])

In [25]:
def recommend_sake(liked_sakes: list, num_recs=5, metric='cosine') -> list:
    
    dists = pairwise_distances(df.iloc[:, 2:], metric=metric)
    dists = pd.DataFrame(data=dist, index=df['name'], columns=df['name'])
    
    sake_summed = dists[liked_sakes].sum(axis=1)
    sake_summed = sake_summed.sort_values(ascending=True)
    
    
    ranked_sakes = sake_summed.index[~sake_summed.index.isin(liked_sakes)]
    ranked_sakes = ranked_sakes.tolist()
    
    recommendations = ranked_sakes[:num_recs]
    
    df_recs = sake.copy()
    df_recs['rec_label'] = np.where(df_recs.name.isin(liked_sakes), 'Liked',
                                   np.where(df_recs.name.isin(recommendations), 'Recommended',
                                           'Other'))
    
    df_recs = df_recs[df_recs.rec_label.isin(['Liked', 'Recommended'])]
    
    return df_recs

In [26]:
sake_i_like = [
               'Dassai 45 Junmai Daiginjo "Otter Festival"', 
               'Kikusui Junmai Ginjo "Chrysanthemum Water"',
               'Shunnoten Tokubetsu Junmai "Fishermen Cup"'
              ]

recommend_sake(sake_i_like, num_recs=3)

,name,url,price,smv,acidity,type,prefecture,kw_word,kw_wines,kw_beer,kw_foods,description,rec_label
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,junmai daiginjo,yamaguchi,chunky,pinot noir chewy whites,ales,mushroom risotto tempura fried chicken caviar ...,this sake has an older brother daiginjo that h...,Liked
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,junmai ginjo,niigata,citrus,sharp whites dry reds,ales ambers,suitable for light and stronger flavored dishe...,true sake owner beau timken recorded kikusui a...,Liked
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,tokubetsu junmai,yamanashi,smooth,pinot noir slippery whites,gentle ales,none,the nose on this awesome looking catch cup is ...,Liked
50,"Tengumai Junmai Daiginjo ""50""",https://www.truesake.com/collections/all/produ...,29,3.0,1.4,junmai daiginjo,ishikawa,velvety,large reds fat whites,none,shellfish sushi creamy dishes grilled fare,the nose on this very flavorful daiginjo is a ...,Recommended
62,"Masumi Okuden Kantsukuri Junmai ""Mirror of Truth""",https://www.truesake.com/collections/all/produ...,30,3.0,1.6,junmai,nagano,waves,dry chardonnay soft reds,pilsners,salmon sauteed in butter tempura hamburgers,okuden has a soft nose of mixed berries melon ...,Recommended
71,"Dassai 45 Nigori ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,nigori junmai daiginjo,yamaguchi,juicy,fruity reds light whites,fruity ales,sparkling wine fare,wow what a tasty nigori that is so flavorful a...,Recommended


In [27]:
recommend_sake(sake_i_like, num_recs=3)

,name,url,price,smv,acidity,type,prefecture,kw_word,kw_wines,kw_beer,kw_foods,description,rec_label
0,"Dassai 45 Junmai Daiginjo ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,junmai daiginjo,yamaguchi,chunky,pinot noir chewy whites,ales,mushroom risotto tempura fried chicken caviar ...,this sake has an older brother daiginjo that h...,Liked
1,"Kikusui Junmai Ginjo ""Chrysanthemum Water""",https://www.truesake.com/collections/all/produ...,33,1.0,1.6,junmai ginjo,niigata,citrus,sharp whites dry reds,ales ambers,suitable for light and stronger flavored dishe...,true sake owner beau timken recorded kikusui a...,Liked
3,"Shunnoten Tokubetsu Junmai ""Fishermen Cup""",https://www.truesake.com/collections/all/produ...,9,3.0,1.6,tokubetsu junmai,yamanashi,smooth,pinot noir slippery whites,gentle ales,none,the nose on this awesome looking catch cup is ...,Liked
50,"Tengumai Junmai Daiginjo ""50""",https://www.truesake.com/collections/all/produ...,29,3.0,1.4,junmai daiginjo,ishikawa,velvety,large reds fat whites,none,shellfish sushi creamy dishes grilled fare,the nose on this very flavorful daiginjo is a ...,Recommended
62,"Masumi Okuden Kantsukuri Junmai ""Mirror of Truth""",https://www.truesake.com/collections/all/produ...,30,3.0,1.6,junmai,nagano,waves,dry chardonnay soft reds,pilsners,salmon sauteed in butter tempura hamburgers,okuden has a soft nose of mixed berries melon ...,Recommended
71,"Dassai 45 Nigori ""Otter Festival""",https://www.truesake.com/collections/all/produ...,30,3.0,1.4,nigori junmai daiginjo,yamaguchi,juicy,fruity reds light whites,fruity ales,sparkling wine fare,wow what a tasty nigori that is so flavorful a...,Recommended
